# Fine Tunning

In [1]:
import torch
import torchvision
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 2.0.0+cu117
torchvision version: 0.15.1+cu117


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchinfo import summary

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Importing Dataset (same as in 04)

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import os
from  pathlib import Path
from PIL import Image

In [8]:
# Create a transforms pipeline manually (required for torchvision < 0.13)
data_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), 
    # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
    # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) 
    # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])
class CustomImageDataset(Dataset):
    def __init__(self, img_dir: str, transform = None, target_transform = None):
        # in case annotations are in a csv file, uncomment below.
        # self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = Path(img_dir)
        self.transform = transform
        self.target_transform = target_transform
        self.paths = list(self.img_dir.glob("*/*.jpg"))
        self._classes = sorted(
            entry.name for entry in os.scandir(self.img_dir) if entry.is_dir()
        )
        self._class_to_idx = {cls_name: i for i, cls_name in enumerate(self._classes)}
        self._idx_to_class = {i: cls_name for i, cls_name in enumerate(self._classes)}

    @property
    def classes(self):
        return self._classes
    @property
    def class_to_idx(self):
        return self._class_to_idx
    @property
    def idx_to_class(self):
        return self._idx_to_class
    
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx: int) -> tuple[Image.Image|torch.Tensor, int|torch.Tensor]:
        image = Image.open(str(self.paths[idx]))
        label = self._class_to_idx[self.paths[idx].parent.name]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [9]:
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"
train_data_custom = CustomImageDataset(img_dir = train_dir, 
                                      transform = data_transform)
test_data_custom = CustomImageDataset(img_dir = test_dir, 
                                     transform = data_transform)
train_data_custom, test_data_custom

(<__main__.CustomImageDataset at 0x7f6e6d0e5480>,
 <__main__.CustomImageDataset at 0x7f6e6d09af80>)

In [11]:
train_dataloader_custom = DataLoader(dataset=train_data_custom, batch_size=32, shuffle=True)
test_dataloader_custom = DataLoader(dataset=test_data_custom, batch_size=32, shuffle=False) 
train_dataloader_custom, test_dataloader_custom

(<torch.utils.data.dataloader.DataLoader at 0x7f6e6d0e4d60>,
 <torch.utils.data.dataloader.DataLoader at 0x7f6e6d0e4fd0>)

In [12]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [13]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [14]:
auto_train_data_custom = CustomImageDataset(img_dir = train_dir, 
                                      transform = auto_transforms)
auto_test_data_custom = CustomImageDataset(img_dir = test_dir, 
                                     transform = auto_transforms)
auto_train_dataloader_custom = DataLoader(dataset=auto_train_data_custom, 
                                          batch_size=32, shuffle=True)
test_dataloader_custom = DataLoader(dataset=auto_test_data_custom, 
                                    batch_size=32, shuffle=False) 